In [1]:
import numpy as np
import pandas as pd
import codecs
import json
import glob
import numpy as np
import pandas as pd
import sys; sys.path.append("/Users/daveyproctor/Documents/Polyspeech/"); from lib.progress import ProgressTracker
import pprint as pp

In [2]:
with open("IBM.key") as fp:
    key = json.load(fp)

In [3]:
from ibm_watson import ToneAnalyzerV3

tone_analyzer = ToneAnalyzerV3(
    version="2016-05-19",
    iam_apikey=key["apikey"],
    url=key["url"]
)

In [4]:
tone_analyzer

In [5]:
text = 'Team, I know that times are tough! Product '\
    'sales have been disappointing for the past three '\
    'quarters. We have a competitive product, but we '\
    'need to do a better job of selling it!'

tone_analysis = tone_analyzer.tone(
    {'text': text},
    content_type='application/json'
).get_result()
print(json.dumps(tone_analysis, indent=2))

{
  "document_tone": {
    "tone_categories": [
      {
        "tones": [
          {
            "score": 0.081788,
            "tone_id": "anger",
            "tone_name": "Anger"
          },
          {
            "score": 0.008589,
            "tone_id": "disgust",
            "tone_name": "Disgust"
          },
          {
            "score": 0.076676,
            "tone_id": "fear",
            "tone_name": "Fear"
          },
          {
            "score": 0.039805,
            "tone_id": "joy",
            "tone_name": "Joy"
          },
          {
            "score": 0.6165,
            "tone_id": "sadness",
            "tone_name": "Sadness"
          }
        ],
        "category_id": "emotion_tone",
        "category_name": "Emotion Tone"
      },
      {
        "tones": [
          {
            "score": 0.829888,
            "tone_id": "analytical",
            "tone_name": "Analytical"
          },
          {
            "score": 0.0,
            "tone_id": "co

In [22]:
df = pd.read_csv("tweetDocsNearElections.csv")

In [23]:
df.head()

,doc_id,doc
0,1,"This morning, I spent some time with area vete..."
1,2,It was nice to see some familiar faces from #W...
2,3,RT @TODAYshow: Joan Rivers 1933-2014 \nhttp://...
3,4,"Thank you, @RepGeneGreen, for helping to prote..."
4,5,RT @timkaine: Pleased @GovernorVA will act to ...


In [24]:
text = df.loc[0,"doc"]

In [25]:
tone_analysis = tone_analyzer.tone(
    {'text': text},
    content_type='application/json'
).get_result()
print(json.dumps(tone_analysis, indent=2))

{
  "document_tone": {
    "tone_categories": [
      {
        "tones": [
          {
            "score": 0.090019,
            "tone_id": "anger",
            "tone_name": "Anger"
          },
          {
            "score": 0.14011,
            "tone_id": "disgust",
            "tone_name": "Disgust"
          },
          {
            "score": 0.054531,
            "tone_id": "fear",
            "tone_name": "Fear"
          },
          {
            "score": 0.638617,
            "tone_id": "joy",
            "tone_name": "Joy"
          },
          {
            "score": 0.482668,
            "tone_id": "sadness",
            "tone_name": "Sadness"
          }
        ],
        "category_id": "emotion_tone",
        "category_name": "Emotion Tone"
      },
      {
        "tones": [
          {
            "score": 0.079945,
            "tone_id": "analytical",
            "tone_name": "Analytical"
          },
          {
            "score": 0.0,
            "tone_id": "c

In [34]:
full=pd.DataFrame()

In [35]:
for toneSet in tone_analysis["document_tone"]["tone_categories"]:
    x = pd.DataFrame(toneSet["tones"]).drop("tone_id", axis=1)
    x["doc_id"] = 1
    full = pd.concat([full,x])
    
#     for tone in toneSet["tones"]:
#         print(tone["score"], tone["tone_name"])


In [36]:
full

,score,tone_name,doc_id
0,0.090019,Anger,1
1,0.140110,Disgust,1
2,0.054531,Fear,1
3,0.638617,Joy,1
4,0.482668,Sadness,1
0,0.079945,Analytical,1
1,0.000000,Confident,1
2,0.031305,Tentative,1
0,0.404867,Openness,1
1,0.889561,Conscientiousness,1


In [28]:
len(df)

1039

In [53]:
full = pd.DataFrame()
doneSet=set()

In [54]:
write_every=5

In [55]:
len(df)

1039

In [48]:
for i in range(len(df)):
    doc_id = df.loc[i, "doc_id"]
    tone_analysis = tone_analyzer.tone(
        {'text': df.loc[i, "doc"]},
        content_type='application/json'
    ).get_result()
    for toneSet in tone_analysis["document_tone"]["tone_categories"]:
        x = pd.DataFrame(toneSet["tones"]).drop("tone_id", axis=1)
        x["doc_id"] = doc_id
        full = pd.concat([full,x])
    if doc_id % write_every == 0:
        print("writing; id={}".format(doc_id))
        full.to_csv("tweetDocsNearElectionsIBM.csv", index=False)
    doneSet.add(doc_id)
full.to_csv("tweetDocsNearElectionsIBM.csv", index=False)

1
writing
2
writing
3
writing
4
writing


In [38]:
full

,score,tone_name,doc_id
0,0.090019,Anger,1
1,0.140110,Disgust,1
2,0.054531,Fear,1
3,0.638617,Joy,1
4,0.482668,Sadness,1
0,0.079945,Analytical,1
1,0.000000,Confident,1
2,0.031305,Tentative,1
0,0.404867,Openness,1
1,0.889561,Conscientiousness,1


In [15]:
dataDir="/Users/daveyproctor/Documents/Polyspeech/data/"
tweetsDF = pd.read_csv(f"{dataDir}tweets/RepAbrahamBefore2018.csv")

,tweet_id,time_created,full_text,is_retweet,retweet_count,favorite_count,hashtag_count,hashtags0,hashtags1,hashtags2,...,user_mentions2,user_mentions3,user_mentions4,in_reply_to_screen_name,in_reply_to_status_id,vader_pos,vader_neg,vader_neu,vader_compound,twitter_account
0,1.059478e+18,2018-11-05 16:09:51,Dianne and I were thrilled to join my granddau...,False,1.0,21.0,1,LA05,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.159,0.000,0.841,0.6249,RepAbraham
1,1.058730e+18,2018-11-03 14:35:58,Today is the day! #GeauxTigers https://t.co/B8...,False,5.0,39.0,1,GeauxTigers,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000,0.000,1.000,0.0000,RepAbraham
2,1.055481e+18,2018-10-25 15:28:13,RT @tmcclelland: Incredibly grateful to give t...,True,1.0,NaN,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.130,0.000,0.870,0.5095,RepAbraham
3,1.055185e+18,2018-10-24 19:50:22,RT @nfib_la: Thank you to our Guardians of Sma...,True,13.0,NaN,0,NaN,NaN,NaN,...,JohnKennedyLA,SteveScalise,RepGarretGraves,NaN,NaN,0.143,0.000,0.857,0.3612,RepAbraham
4,1.055185e+18,2018-10-24 19:49:51,RT @ScottFranklinJr: Incredible tour of the Ir...,True,9.0,NaN,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.127,0.000,0.873,0.4939,RepAbraham
5,1.053811e+18,2018-10-21 00:52:54,Great seeing my friends @RepMikeJohnson and hi...,False,7.0,27.0,2,HBTD,LA05,NaN,...,NaN,NaN,NaN,NaN,NaN,0.298,0.000,0.702,0.8020,RepAbraham
6,1.053341e+18,2018-10-19 17:42:10,"Democrats want to abolish ICE, but I like that...",False,6.0,30.0,1,LA05,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.166,0.156,0.678,-0.1901,RepAbraham
7,1.052958e+18,2018-10-18 16:21:29,Thanks Gen. Frank for inviting me to participa...,False,3.0,14.0,1,LA05,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.183,0.107,0.711,0.5423,RepAbraham
8,1.052600e+18,2018-10-17 16:40:05,#LA05 is the largest recipient ($1.12 million)...,False,3.0,12.0,1,LA05,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.034,0.000,0.966,0.0772,RepAbraham
9,1.050402e+18,2018-10-11 15:03:50,.@4H Clubs help our kids grow as people and le...,False,2.0,12.0,2,National4HWeek,LA05,NaN,...,NaN,NaN,NaN,NaN,NaN,0.234,0.000,0.766,0.9057,RepAbraham


In [20]:
"".join(["asdf asdf","asdf fdf"])

'asdf asdfasdf fdf'

In [40]:
text = "".join(tweetsDF.loc[0:40,"full_text"])
text

'Dianne and I were thrilled to join my granddaughter’s kindergarten class today. Dianne read “House Mouse, Senate Mouse” to introduce them to how our system of government works. #LA05 https://t.co/EZh8IcTJN3Today is the day! #GeauxTigers https://t.co/B8ixSAMOUhRT @tmcclelland: Incredibly grateful to give the game ball this past Saturday to Colonel (Dr.) Chris Hudson, Commandor of the 2d Medical Gr…RT @nfib_la: Thank you to our Guardians of Small Business @BillCassidy @JohnKennedyLA @SteveScalise @RepGarretGraves @RepAbraham @RepMikeJo…RT @ScottFranklinJr: Incredible tour of the Iraq rice shipment out of Lake Charles.  @RepAbraham wanted to get a first hand look at his har…Great seeing my friends @RepMikeJohnson and his wife Kelly and Tech XXII at @LATech homecoming today. #HBTD #LA05 https://t.co/PgMCE1xruCDemocrats want to abolish ICE, but I like that these agents are taking dangerous child predators off of Louisiana streets. #LA05 https://t.co/KIKSApGaC5Thanks Gen. Frank for inviting

In [41]:

tone_analysis = tone_analyzer.tone(
    {'text': text},
    content_type='application/json'
).get_result()
print(json.dumps(tone_analysis, indent=2))

{
  "document_tone": {
    "tone_categories": [
      {
        "tones": [
          {
            "score": 0.082845,
            "tone_id": "anger",
            "tone_name": "Anger"
          },
          {
            "score": 0.117989,
            "tone_id": "disgust",
            "tone_name": "Disgust"
          },
          {
            "score": 0.092857,
            "tone_id": "fear",
            "tone_name": "Fear"
          },
          {
            "score": 0.646884,
            "tone_id": "joy",
            "tone_name": "Joy"
          },
          {
            "score": 0.507031,
            "tone_id": "sadness",
            "tone_name": "Sadness"
          }
        ],
        "category_id": "emotion_tone",
        "category_name": "Emotion Tone"
      },
      {
        "tones": [
          {
            "score": 0.023579,
            "tone_id": "analytical",
            "tone_name": "Analytical"
          },
          {
            "score": 0.0,
            "tone_id": "

In [42]:
len(tone_analysis["sentences_tone"])

97